<a href="https://colab.research.google.com/github/GauravSahani1417/PySpark-Basic-Implementation/blob/main/PySpark_Transformations_and_Actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setting up the PySpark environment

# Install java 8
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Apache Spark binary: This link can change based on the version. Update this link with the latest version before using
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz

# Unzip file
!tar -xf spark-3.0.2-bin-hadoop2.7.tgz

# Install findspark: Adds Pyspark to sys.path at runtime
!pip install -q findspark

# Install pyspark
!pip install pyspark

# Add environmental variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

# findspark will locate spark in the system
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,185 kB]
Hit:12 http://ppa.launchpad.net/cra

In [ ]:
#Initialize Spark Session

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Hands-on PySpark on Google Colab") \
        .getOrCreate()

In [ ]:
spark

In [ ]:
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv -P sample_data/

In [ ]:
# We can set header='true' and inferSchema='true' to infer the schema while reading the data

filepath = "sample_data/in-vehicle-coupon-recommendation.csv"
spark_df = spark.read.format('csv').options(header='true', inferSchema='true').load(filepath)
spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+----------+------+---+-----------------+------------+------------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|destination    |passanger|weather|temperature|time|coupon               |expiration|gender|age|maritalStatus    |has_children|education               |occupation|income         |car |Bar  |CoffeeHouse|CarryAway|RestaurantLessThan20|Restaurant20To50|toCoupon_GEQ5min|toCoupon_GEQ15min|toCoupon_GEQ25min|direction_same|direction_opp|Y  |
+---------------+---------+-------+-----------+----+---------------------+----------+------+---+-----------------+------------+------------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+--------

Transformations and Actions

In [ ]:
from pyspark.sql import functions as F

In [ ]:
#Actions

# 1.SHOW:
columns_to_use = ["destination", "passanger", "weather", "temperature", "time", "coupon", "gender", "age", "has_children", "income", "Y"]
spark_df = spark_df.select(*columns_to_use)
spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |
+---------------+---------+-------+-----------+-

In [ ]:
#count: count the number of rows in a dataframe
#We did "df.shape" in pandas

spark_df.count(), len(spark_df.columns)

(12684, 11)

In [ ]:
#collect: Run action command to compute and get the result

spark_df.agg(F.min("temperature"))

DataFrame[min(temperature): int]

In [ ]:
spark_df.agg(F.min("temperature")).show()

+----------------+
|min(temperature)|
+----------------+
|              30|
+----------------+



In [ ]:
spark_df.agg(F.max("temperature")).show()

+----------------+
|max(temperature)|
+----------------+
|              80|
+----------------+



In [ ]:
spark_df.collect()[:5]

[Row(destination='No Urgent Place', passanger='Alone', weather='Sunny', temperature=55, time='2PM', coupon='Restaurant(<20)', gender='Female', age='21', has_children=1, income='$37500 - $49999', Y=1),
 Row(destination='No Urgent Place', passanger='Friend(s)', weather='Sunny', temperature=80, time='10AM', coupon='Coffee House', gender='Female', age='21', has_children=1, income='$37500 - $49999', Y=0),
 Row(destination='No Urgent Place', passanger='Friend(s)', weather='Sunny', temperature=80, time='10AM', coupon='Carry out & Take away', gender='Female', age='21', has_children=1, income='$37500 - $49999', Y=1),
 Row(destination='No Urgent Place', passanger='Friend(s)', weather='Sunny', temperature=80, time='2PM', coupon='Coffee House', gender='Female', age='21', has_children=1, income='$37500 - $49999', Y=0),
 Row(destination='No Urgent Place', passanger='Friend(s)', weather='Sunny', temperature=80, time='2PM', coupon='Coffee House', gender='Female', age='21', has_children=1, income='$375

In [ ]:
#Transformations

spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |
+---------------+---------+-------+-----------+-

In [ ]:
#distinct: Get unique values in a columns

spark_df.select("gender").distinct().show()

+------+
|gender|
+------+
|Female|
|  Male|
+------+



In [ ]:
spark_df.select("passanger").distinct().show()

+---------+
|passanger|
+---------+
|  Partner|
|    Alone|
|Friend(s)|
|   Kid(s)|
+---------+



In [ ]:
#withColumn: Create new columns & withColumnRenamed

#We will add extra column "Constant column" and fill it with "full data"
updated_spark_df = spark_df.withColumn("constant_column", F.lit("full_data"))
updated_spark_df.show(3, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |constant_column|
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |full_data      |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |full_data      |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |full_data      |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+
only showing top 3 rows



In [ ]:
#We will use "When" function to replace our Gender column as "1 for Male" & "0 for Female"
updated_spark_df = updated_spark_df.withColumn("gender_mapped", F.when(F.col("gender") == "Female", 0)\
                                                                  .when(F.col("gender") == "Male", 1)\
                                                                  .otherwise(2))
updated_spark_df.sample(0.2).show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+-------------+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |constant_column|gender_mapped|
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+-------------+
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |full_data      |0            |
|No Urgent Place|Friend(s)|Sunny  |55         |2PM |Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |full_data      |0            |
|Home           |Alone    |Sunny  |80         |6PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |full_data      |0            |
|Work           |Alone    |Sunny  |80         |7AM |Carry out & Take away|Female|21 |1           |$37500 -

In [ ]:
#We will use withColumnRenamed to rename our column
updated_spark_df = updated_spark_df.withColumnRenamed("passanger", "passenger")
updated_spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+-------------+
|destination    |passenger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |constant_column|gender_mapped|
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+-------------+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |full_data      |0            |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |full_data      |0            |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |full_data      |0            |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 -

In [ ]:
updated_spark_df = updated_spark_df.withColumnRenamed("constant_column", "const_col")
updated_spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------+-------------+
|destination    |passenger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |const_col|gender_mapped|
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------+-------------+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |full_data|0            |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |full_data|0            |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |full_data|0            |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |full_data|0            |

In [ ]:
#filter: Filter data

updated_spark_df.select("temperature").distinct().show()

+-----------+
|temperature|
+-----------+
|         55|
|         80|
|         30|
+-----------+



In [ ]:
temp_80_df = updated_spark_df.filter(F.col("temperature") == 80)
temp_80_df.show(3)

+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
|    destination|passenger|weather|temperature|time|              coupon|gender|age|has_children|         income|  Y|const_col|gender_mapped|
+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|        Coffee House|Female| 21|           1|$37500 - $49999|  0|full_data|            0|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|Carry out & Take ...|Female| 21|           1|$37500 - $49999|  1|full_data|            0|
|No Urgent Place|Friend(s)|  Sunny|         80| 2PM|        Coffee House|Female| 21|           1|$37500 - $49999|  0|full_data|            0|
+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
only s

In [ ]:
updated_spark_df.count(), temp_80_df.count()

(12684, 6528)

In [ ]:
temp_45_90_df = updated_spark_df.filter((F.col("temperature") > 45) & (F.col("temperature") < 90))
temp_45_90_df.show(3)

+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
|    destination|passenger|weather|temperature|time|              coupon|gender|age|has_children|         income|  Y|const_col|gender_mapped|
+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
|No Urgent Place|    Alone|  Sunny|         55| 2PM|     Restaurant(<20)|Female| 21|           1|$37500 - $49999|  1|full_data|            0|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|        Coffee House|Female| 21|           1|$37500 - $49999|  0|full_data|            0|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|Carry out & Take ...|Female| 21|           1|$37500 - $49999|  1|full_data|            0|
+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
only s

In [ ]:
#Groupby and aggregate
spark_df.show(3, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
only showing top 3 rows



In [ ]:
spark_df.groupby("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|Female| 6511|
|  Male| 6173|
+------+-----+



In [ ]:
spark_df.groupby("temperature").count().show()

+-----------+-----+
|temperature|count|
+-----------+-----+
|         55| 3840|
|         80| 6528|
|         30| 2316|
+-----------+-----+



In [ ]:
spark_df.groupby("gender").agg(F.min("temperature"), F.max("temperature")).show()

+------+----------------+----------------+
|gender|min(temperature)|max(temperature)|
+------+----------------+----------------+
|Female|              30|              80|
|  Male|              30|              80|
+------+----------------+----------------+



In [ ]:
spark_df.groupby("gender").agg(F.min("temperature").alias("min_temperature"), F.max("temperature").alias("max_temperature")).show()

+------+---------------+---------------+
|gender|min_temperature|max_temperature|
+------+---------------+---------------+
|Female|             30|             80|
|  Male|             30|             80|
+------+---------------+---------------+

